In [71]:
import chromadb
from elasticsearch import Elasticsearch
import fitz
import os
import numpy as np

import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, Text, LargeBinary
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import declarative_base
from sqlalchemy import MetaData
from sqlalchemy import select

import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [13]:
Base = declarative_base()

class Document(Base):
    __tablename__ = 'documents'
    id = Column(Integer, primary_key=True)
    title = Column(String)
    content = Column(Text)
    embedding = Column(LargeBinary, nullable=True)

In [6]:
engine = create_engine('sqlite:///data.db')

In [7]:
Base.metadata.create_all(engine)

In [54]:
Session = sessionmaker(bind=engine)
session = Session()

In [76]:
documents = [
    ("Document 1", "О порядке расчета размера участия иностранного капитала в совокупном уставном капитале кредитных организаций, имеющих лицензию на осуществление банковских операций. На основании статьи 18 Федерального закона «О банках и банковской деятельности» (в редакции Федерального закона от 3 февраля 1996 года № 17-ФЗ) (Ведомости Съезда народных депутатов РСФСР и Верховного Совета РСФСР, 1990, № 27, ст. 357; Собрание законодательства Российской Федерации, 1996, № 6, ст. 492; 1998, № 31, ст. 3829; 1999, № 28, ст. 3459, ст. 3469; 2001, № 26, ст. 2586; № 33, ст. 3424; 2002, № 12, ст. 1093; 2003, № 27,  ст. 2700; № 50, ст. 4855; № 52, ст. 5033, ст. 5037; 2004, № 27, ст. 2711; № 31, ст. 3233; 2005, № 1, ст. 18, ст. 45; № 30, ст. 3117; 2006, № 6, ст. 636; № 19, ст. 4011; № 41, ст. 4845; № 45, ст. 5425; № 50, ст. 6238; 2008, № 10, ст. 895; 2009, № 1, ст. 23; № 9, ст. 1043; № 18, ст. 2153; № 23, ст. 2776; № 30, ст. 3739; № 48, ст. 5731; № 52, ст. 6428; 2010, № 8, ст. 775; № 27, ст. 3432; № 30, ст. 4012; № 31, ст. 4193; № 47, ст. 6028; 2011, № 7, ст. 905; № 27, ст. 3873, ст. 3880; № 29, ст. 4291; № 48, ст. 6728, ст. 6730; № 49, ст. 7069; № 50, ст. 7351; 2012, № 27, ст. 3588; № 31, ст. 4333; № 50, ст. 6954; № 53, ст. 7605, ст. 7607; 2013, № 11, ст. 1076; № 19, ст. 2317, ст. 2329; № 26, ст. 3207; № 27, ст. 3438, ст. 3477; № 30, ст. 4048; № 40, ст. 5036; № 49, ст. 6336; № 51, ст. 6683, ст. 6699; 2014, № 6, ст. 563; № 19, ст. 2311; № 26, ст. 3379, ст. 3395; № 30, ст. 4219; № 40, ст. 5317, ст. 5320; № 45, ст. 6144, ст. 6154; № 49, ст. 6912; № 52, ст. 7543; 2015, № 1, ст. 37; № 17, ст. 2473; № 27, ст. 3947, ст. 3950; № 29, ст. 4355, ст. 4385; № 51, ст. 7243; «Официальный интернетпортал правовой информации» (www.pravo.gov;ru), 29 декабря 2015 года) (далее - Федеральный закон «О банках и банковской деятельности») Банк России устанавливает порядок расчета размера участия иностранного капитала в совокупном уставном капитале кредитных организаций, имеющих лицензию на осуществление банковских операций (далее - размер участия иностранного капитала в совокупном уставном капитале кредитных организаций). 1. Размер участия иностранного капитала в совокупном уставном капитале кредитных организаций рассчитывается Банком России (Департаментом лицензирования деятельности и финансового оздоровления кредитных организаций) на основе имеющихся в Банке России сведений об оплаченных акциях (долях) кредитных организаций и их акционерах (участниках), в том числе данных о номинальной стоимости принадлежащих нерезидентам акций (долей) кредитных организаций, отраженных в списках акционеров (участников) кредитных организаций и уведомлениях о приобретении более одного процента акций (долей) кредитных организаций, представляемых в Банк России в соответствии с Инструкцией Банка России от 2 апреля 2010 года № 135-И «О порядке принятия Банком России решения о государственной регистрации кредитных организаций и выдаче лицензий на осуществление банковских операций», зарегистрированной Министерством юстиции Российской Федерации 22 апреля 2010 года № 16965, 17 декабря • № 21869, 16 декабря 2011 года № 22645, 5 ноября 2013 года № 30308, 25 декабря 2013 года № 30818, 11 декабря 2014 года № 35134, 15 июня 2015 года № 37658 («Вестник Банка России» от 30 апреля 2010 года № 23, от 3 О декабря 201 О года № 73, от 22 июня 2011 года № 3 3, от 28 сентября 2011 года № 54, от 21 декабря 2011 года № 72, от 13 ноября 2013 года № 61, от 20 января 2014 года № 5 - 6, от 22 декабря 2014 года № 112, от 22 июня 2015 года № 54). В целях настоящего Указания под оплаченными акциями (долями) кредитных организаций понимаются: акции кредитной организации, если на дату расчета размера участия иностранного капитала в совокупном уставном капитале кредитных организаций такие акции размещены и сведения о соответствующей величине уставного капитала кредитной организации по итогам размещения внесены в единый государственный реестр юридических лиц, а также акции выпуска (дополнительного выпуска) кредитной организации, государственная регистрация отчета об итогах которого осуществлена или уведомление об итогах которого получено Банком России в порядке, установленном Инструкцией Банка России от 27 декабря 2013 года № 148-И «О порядке осуществления процедуры эмиссии ценных бумаг кредитных организаций на территории Российской Федерации», зарегистрированной Министерством юстиции Российской Федерации 28 февраля 2014 года № 31458, 10 декабря 2014 года № 35118, 2 марта 2015 года № 36322, 28 июля 2015 года № 38242 («Вестник Банка России» от 28 марта 2014 года № 32 - 33, от 22 декабря 2014 года № 112, от 18 марта 2015 года № 22, от 12 августа 2015 года № 66); доли кредитной организации, если на дату расчета размера участия иностранного капитала в совокупном уставном капитале кредитных организаций сведения о соответствующей величине уставного капитала кредитной организации внесены в единый государственный реестр юридических лиц. 2. Размер участия иностранного капитала в совокупном уставном капитале кредитных организаций рассчитывается как соотношение иностранных инвестиций нерезидентов в уставные капиталы кредитных организаций, имеющих лицензию на осуществление банковских операций, и совокупного уставного капитала кредитных организаций, имеющих лицензию на осуществление банковских операций, по формуле: Q = (La * lOO%)/Qук, где: Q - размер участия иностранного капитала в совокупном уставном капитале кредитных организаций; La - иностранные инвестиции нерезидентов в уставные капиталы кредитных организаций, имеющих лицензию на осуществление банковских операций, за исключением иностранных инвестиций, указанных в части второй статьи 18 Федерального закона «О банках и банковской деятельности»; Qук - совокупный уставный капитал кредитных организаций, имеющих лицензию на осуществление банковских операций. 3. Настоящее Указание вступает в силу по истечении 1 О дней после дняего официального опубликования. Председатель Центрального банка Российской Федерации Э.С. Набиуллина"),
    ("Document 2", "О порядке предоставления конкурсным управляющим отчетов, а также иной информации о ходе конкурсного производства в отношении негосударственного пенсионного фонда, осуществляющего деятельность по обязательному пенсионному страхованию, и требованиях к их составу Настоящее Указание на основании абзаца второго пункта 1О статьи 1877 Федерального закона от 26 октября 2002 года № 127-ФЗ «О несостоятельности (банкротстве)» (Собрание законодательства Российской Федерации, 2002, № 43, ст. 4190; 2004, № 35, ст. 3607; 2005, № 1, ст. 18, ст. 46; № 44, ст. 4471; 2006, № 30, ст. 3292; № 52, ст. 5497; 2007, № 7, ст. 834; № 18, ст. 2117; № 30, ст. 3754; № 41, ст. 4845; № 49, ст. 6079; 2008, № 30, ст. 3616; № 49, ст. 5748; 2009, № 1, ст. 4, ст. 14; № 18, ст. 2153; № 29, ст. 3632; № 51, ст. 6160; № 52, ст. 6450; 2010, № 17, ст. 1988; № 31, ст. 4188, ст. 4196; 2011, № 1, ст. 41; № 7, ст. 905; № 19, ст. 2708; № 27, ст. 3880; № 29, ст. 4301; № 30, ст. 4576; № 48, ст. 6728; № 49, ст. 7015, ст. 7024, ст. 7040, т. 7061, ст. 7068; № 50, ст. 7351, ст. 7357; 2012, № 31, ст. 4333; № 53, ст. 7607,  ст. 7619; 2013, № 23, ст. 2871; № 26, ст. 3207; № 27, ст. 3477, ст. 3481; № 30, ст. 4084; № 51, ст. 6699; № 52, ст. 6975, ст. 6984; 2014, № 11, ст. 1095, ст. 1098; № 30, ст. 4217; № 49, ст. 6914; № 52, ст. 7543; 2015, № 1, ст. 10, ст. 11, ст. 35; № 27, ст. 3495, ст. 3958, ст. 3967, ст. 3977; № 29, ст. 4350, ст. 4355, ст. 4362) (далее - Федеральный закон «О несостоятельности (банкротстве)») устанавливает порядок предоставления конкурсным управляющим в Банк России отчетов, а также иной информации о ходе конкурсного производства в отношении негосударственного пенсионного фонда, осуществляющего деятельность по обязательному пенсионному страхованию (далее - фонд), и требования к их составу. Отчет конкурсного управляющего о своей деятельности не позднее 30 дней со дня его рассмотрения собранием кредиторов или комитетом кредиторов направляется в Банк России и должен содержать следующую информацию: о сформированной конкурсной массе, средствах пенсионных накоплений и средствах пенсионных резервов, в том числе о ходе и (или) об итогах инвентаризации имущества, о ходе и (или) результатах оценки имущества в случае привлечения оценщика для оценки такого имущества; о размере денежных средств, поступивших на счета фонда, открытых в государственной корпорации  «Агентство  по страхованию  вкладов» (далее - Агентство), и об источниках данных поступлений; о ходе реализации имущества фонда, с указанием сумм, поступивших от реализации имущества; о	количестве	и	об	общем	размере требований	о	взыскании задолженности, предъявленных конкурсным управляющим к третьим лицам; о предпринятых мерах по обеспечению сохранности имущества фонда, а также по выявлению и истребованию имущества фонда, находящегося во владении у третьих лиц; о предпринятых мерах по признанию сделок фонда недействительными, по заявлению отказа от исполнения договоров фонда; о ведении реестра требований кредиторов с указанием общего размера  требований  кредиторов, включенных  в этот  реестр,  и отдельно - относительно каждой очереди; о количестве работников фонда, продолжающих свою деятельность в ходе конкурсного производства, а также о количестве работников фонда, уволенных (сокращенных) в ходе конкурсного производства; о проведенной конкурсным управляющим работе по закрытию счетов фонда и ее результатах; о сумме расходов на проведение конкурсного производства с указанием их назначения; о привлечении к субсидиарной ответственности лиц, которые в соответствии с законодательством Российской Федерации несут такую ответственность по обязательствам фонда в связи с доведением его до банкротства. Отчет  о  результатах  проведения  конкурсного  производства одновременно с направлением в арбитражный суд предоставляется в Банк России и должен содержать следующую информацию: данные о количестве и сумме требований кредиторов, включенных в реестр требований кредиторов, удовлетворенных в ходе конкурсного производства; о  размере  поступления  денежных  средств  в  ходе  конкурсного производства и источниках поступления; информацию о принятых мерах, направленных на выявление признаков преднамеренного и фиктивного банкротства. Конкурсный	управляющий	в	ходе	конкурсного производства предоставляет в Банк России следующую информацию. Уведомление о назначении представителя конкурсного управляющего (далее - представитель Агентства) в деле о банкротстве или о назначении нового представителя Агентства в течение 1О дней со дня вынесения арбитражным судом решения о признании фонда банкротом и об открытии конкурсного производства либо возникновения обстоятельств, влекущих невозможность осуществления представителем Агентства своих полномочий, в том числе вследствие его отстранения от исполнения обязанностей. Уведомление о внесении в Единый государственный реестр юридических лиц записи о том, что фонд находится в процессе принудительной ликвидации (направляется конкурсным управляющим в течение трех рабочих дней со дня принятия арбитражным судом соответствующего решения). Уведомление о закрытии филиалов и представительств фонда (направляется конкурсным управляющим в течение трех рабочих дней со дня принятия им решения о закрытии соответствующего структурного подразделения). Уведомление  о включении требований Банка России в реестр требований кредиторов (направляется конкурсным управляющим в течение трех рабочих дней со дня внесения указанных сведений в реестр требований кредиторов). Тексты публикаций о ходе мероприятий конкурсного производства в отношении фондов, для их размещения в «Вестнике Банка России» в соответствии с пунктом 14 статьи 1877 Федерального закона «О несостоятельности (банкротстве)», в том числе сведения: о проведении собрания (комитета) кредиторов; о продаже имущества и результатах торгов имуществом фонда; о принятом Агентством по согласованию с Банком России решении о передаче обязанности по выплате пожизненных негосударственных пенсий и средств пенсионных резервов другому негосударственному пенсионному фонду; о прекращении производства по делу о банкротстве; об удовлетворении заявлений третьих лиц о намерении погасить обязательства фонда. Определение арбитражного суда о завершении конкурсного производства либо о прекращении конкурсного производства (направляется конкурсным управляющим в срок не позднее 30 дней со дня вынесения соответствующего определения арбитражного суда).Иная информация о ходе конкурсного производства, предоставляемая в случае необходимости уточнения информации, предусмотренной настоящим Указанием, в срок не позднее 15 рабочих дней с даты получения запроса Банка России. Конкурсный управляющий предоставляет в Банк России отчеты и иную информацию о ходе конкурсного производства путем направления электронных документов, подписанных усиленной квалифицированной электронной подписью в соответствии с требованиями Федерального закона от 6 апреля 2011 года № 63-ФЗ «Об электронной подписи» (Собрание законодательства Российской  Федерации,  2011,  №  15,  ст.  2036;  №  27, ст. 3880; 2012, № 29, ст. 3988; 2013, № 14, ст. 1668; № 27, ст. 3463, ст. 3477; 2014, № 11, ст. 1098; № 26, ст. 3390), по телекоммуникационным каналам связи, в том числе через информационно-телекоммуникационную сеть «Интернет», либо при отсутствии технической возможности на бумажных носителях. Документ, представляемый на бумажном  носителе, объем которого превышает один лист, должен быть прошит, пронумерован, содержать заверительную надпись конкурсного управляющего. Настоящее Указание вступает в силу по истечении 1О дней после дня его официального опубликования. Председатель Центрального банка Российской Федерации Э.С. Набиуллина.")
]

In [77]:
def save_document(title, content):
    """
    Saves a document to the database.
    """
    document = Document(title=title, content=content)
    session.add(document)
    session.commit()

In [79]:
for title, content in documents:
    save_document(title, content)

In [80]:
def load_and_split_documents() -> list[dict]:
    """
    Loads documents from the database and splits them into chunks.
    Returns a list of dictionaries.
    """
    # Загружаем документы
    documents = session.query(Document).all()

    # Разбиваем документы на фрагменты
    final_docs = []
    for doc in documents:
        # Split content into chunks
        chunks = split_into_chunks(doc.content)

        # Create a dictionary for each chunk
        for i, chunk in enumerate(chunks):
            chunk_dict = {
                "id": f"{doc.id}_{i}",
                "content": chunk,
                "sourcefile": doc.title,
            }
            final_docs.append(chunk_dict)

    return final_docs

def split_into_chunks(text, chunk_size=500, chunk_overlap=100):
    """
    Splits the text into chunks.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start = end - chunk_overlap
        if end == len(text):
            break
    return chunks

# Пример
result = load_and_split_documents()
print(result)

[{'id': '1_0', 'content': 'О порядке расчета размера участия иностранного капитала в совокупном уставном капитале кредитных организаций, имеющих лицензию на осуществление банковских операций. На основании статьи 18 Федерального закона «О банках и банковской деятельности» (в редакции Федерального закона от 3 февраля 1996 года № 17-ФЗ) (Ведомости Съезда народных депутатов РСФСР и Верховного Совета РСФСР, 1990, № 27, ст. 357; Собрание законодательства Российской Федерации, 1996, № 6, ст. 492; 1998, № 31, ст. 3829; 1999, № 28', 'sourcefile': 'Document 1'}, {'id': '1_1', 'content': 'Собрание законодательства Российской Федерации, 1996, № 6, ст. 492; 1998, № 31, ст. 3829; 1999, № 28, ст. 3459, ст. 3469; 2001, № 26, ст. 2586; № 33, ст. 3424; 2002, № 12, ст. 1093; 2003, № 27,  ст. 2700; № 50, ст. 4855; № 52, ст. 5033, ст. 5037; 2004, № 27, ст. 2711; № 31, ст. 3233; 2005, № 1, ст. 18, ст. 45; № 30, ст. 3117; 2006, № 6, ст. 636; № 19, ст. 4011; № 41, ст. 4845; № 45, ст. 5425; № 50, ст. 6238; 200

In [81]:
tagged_data = [TaggedDocument(words=word_tokenize(doc['content'].lower()), tags=[str(doc['id'])]) for doc in result]

In [82]:
max_epochs = 100
vec_size = 100
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm=1)  # Используем Distributed Memory (PV-DM)

model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print(f'Эпоха {epoch}')
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # Уменьшение скорости обучения с каждой эпохой
    model.alpha -= 0.0002
    # Фиксирование скорости обучения, не позволяя ей стать ниже минимального значения
    model.min_alpha = model.alpha

Эпоха 0
Эпоха 1
Эпоха 2
Эпоха 3
Эпоха 4
Эпоха 5
Эпоха 6
Эпоха 7
Эпоха 8
Эпоха 9
Эпоха 10
Эпоха 11
Эпоха 12
Эпоха 13
Эпоха 14
Эпоха 15
Эпоха 16
Эпоха 17
Эпоха 18
Эпоха 19
Эпоха 20
Эпоха 21
Эпоха 22
Эпоха 23
Эпоха 24
Эпоха 25
Эпоха 26
Эпоха 27
Эпоха 28
Эпоха 29
Эпоха 30
Эпоха 31
Эпоха 32
Эпоха 33
Эпоха 34
Эпоха 35
Эпоха 36
Эпоха 37
Эпоха 38
Эпоха 39
Эпоха 40
Эпоха 41
Эпоха 42
Эпоха 43
Эпоха 44
Эпоха 45
Эпоха 46
Эпоха 47
Эпоха 48
Эпоха 49
Эпоха 50
Эпоха 51
Эпоха 52
Эпоха 53
Эпоха 54
Эпоха 55
Эпоха 56
Эпоха 57
Эпоха 58
Эпоха 59
Эпоха 60
Эпоха 61
Эпоха 62
Эпоха 63
Эпоха 64
Эпоха 65
Эпоха 66
Эпоха 67
Эпоха 68
Эпоха 69
Эпоха 70
Эпоха 71
Эпоха 72
Эпоха 73
Эпоха 74
Эпоха 75
Эпоха 76
Эпоха 77
Эпоха 78
Эпоха 79
Эпоха 80
Эпоха 81
Эпоха 82
Эпоха 83
Эпоха 84
Эпоха 85
Эпоха 86
Эпоха 87
Эпоха 88
Эпоха 89
Эпоха 90
Эпоха 91
Эпоха 92
Эпоха 93
Эпоха 94
Эпоха 95
Эпоха 96
Эпоха 97
Эпоха 98
Эпоха 99


In [84]:
document_vectors = []
for doc in result:
    doc_vector = {
        "id": doc['id'].lower()
        "emb": model.infer_vector(word_tokenize(doc['content'].lower()))
    }
    document_vectors.append(doc_vector)

In [85]:
print(document_vectors)

[{'id': '1_0', 'emb': array([-0.06274775, -0.10623313, -0.08241628, -0.00418281, -0.13486528,
       -0.3269706 ,  0.12797302,  0.34329277, -0.25784695, -0.05701515,
       -0.04900191, -0.26679313, -0.10674293,  0.11109606,  0.02323827,
       -0.18315221,  0.15255852, -0.09743233, -0.16619684, -0.2544843 ,
        0.161564  , -0.01078735,  0.20372157,  0.12620929,  0.02912333,
        0.09125273, -0.1893581 ,  0.00596876, -0.204961  , -0.09289054,
        0.24716903,  0.23806103,  0.07728685, -0.11648387,  0.13636896,
        0.17499095,  0.03282935, -0.20326048, -0.01318483, -0.12723699,
        0.02610978, -0.18100046, -0.21145648, -0.1270468 ,  0.08500011,
        0.04813832, -0.08902109,  0.07620678,  0.09101744,  0.09279656,
       -0.0561472 , -0.13131368,  0.06588626, -0.14235543, -0.15761758,
        0.16921179,  0.02450226, -0.06377467, -0.24854419,  0.04562291,
        0.08943737,  0.13285574,  0.01560444, -0.07118223, -0.03812012,
        0.25626254,  0.19867502,  0.319223

In [78]:
session.close()

In [90]:
def find_most_relevant_chunks(query_text, top_n=5):
    # Оцениваем текст запроса с помощью модели Doc2Vec
    query_vector = model.infer_vector(word_tokenize(query_text.lower()))
    
    # Вычисляем сходство между запросом и чанками
    similarities = []
    for chunk_vector in document_vectors:
        similarity = np.dot(query_vector, chunk_vector['emb'])  # Вычисляем скалярное произведение
        similarities.append((chunk_vector, similarity))
    
    # Сортируем чанки по сходству
    sorted_chunks = sorted(similarities, key=lambda x: x[1], reverse=True)
    
    # Возвращаем топ N наиболее релевантных чанков
    return sorted_chunks[:top_n]


In [97]:
query_text = "Кто устанавливает порядок расчета размера участия иностранного капитала?"
top_chunks = find_most_relevant_chunks(query_text)
for chunk, similarity in top_chunks:
    id_from_chunk = chunk['id']
   matching_result = next((doc for doc in result if doc['id'] == id_from_chunk), None)
    if matching_result:
        # Если объект найден, выводим его содержимое 'content' и сходство
        content = matching_result['content']
        print(f"Сходство: {similarity}, Содержимое: {content}")
    else:
        print(f"Для чанка с id={id_from_chunk} не найдено соответствующего объекта в массиве result.")

Сходство: 0.708401620388031, Содержимое: 015 года № 37658 («Вестник Банка России» от 30 апреля 2010 года № 23, от 3 О декабря 201 О года № 73, от 22 июня 2011 года № 3 3, от 28 сентября 2011 года № 54, от 21 декабря 2011 года № 72, от 13 ноября 2013 года № 61, от 20 января 2014 года № 5 - 6, от 22 декабря 2014 года № 112, от 22 июня 2015 года № 54). В целях настоящего Указания под оплаченными акциями (долями) кредитных организаций понимаются: акции кредитной организации, если на дату расчета размера участия иностранного капитала в сово
Сходство: 0.665962278842926, Содержимое: ытии соответствующего структурного подразделения). Уведомление  о включении требований Банка России в реестр требований кредиторов (направляется конкурсным управляющим в течение трех рабочих дней со дня внесения указанных сведений в реестр требований кредиторов). Тексты публикаций о ходе мероприятий конкурсного производства в отношении фондов, для их размещения в «Вестнике Банка России» в соответствии с пунктом 14

In [98]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration 
tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-1.7B', eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('ai-forever/FRED-T5-1.7B')
device = torch.device('cpu')
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(50364, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50364, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1536, out_features=1536, bias=False)
              (k): Linear(in_features=1536, out_features=1536, bias=False)
              (v): Linear(in_features=1536, out_features=1536, bias=False)
              (o): Linear(in_features=1536, out_features=1536, bias=False)
              (relative_attention_bias): Embedding(32, 24)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1536, out_features=4096, bias=False)
              (wi_1): Linear(in_features=1536, out_features=4096, bias=False)
       

In [102]:
def get_chunk_contents(chunk_ids, result):
    chunk_contents = []
    for chunk_id in chunk_ids:
        for doc in result:
            if doc['id'] == chunk_id:
                chunk_contents.append(doc['content'])
                break  # Прерываем цикл после нахождения соответствия
    return chunk_contents

In [111]:
def generate_response(query_text, top_chunks):
    # Получаем только 'id' чанков из top_chunks
    chunk_ids = [chunk['id'] for chunk, _ in top_chunks]

    # Получаем содержимое чанков, используя функцию get_chunk_contents
    chunk_contents = get_chunk_contents(chunk_ids, result)

    # Объединяем текст запроса с содержимым чанков
    combined_text = " Вспомогательный текст: ".join(chunk_contents)
    combined_text = f"Вопрос: {query_text}. Вспомогательный текст: {combined_text}"
    print("Текст запроса:[ " + combined_text + " ]")

    # Подготовим входные данные для модели
    input_ids = torch.tensor([tokenizer.encode('<LM>' + combined_text)]).to(device)

    # Генерация ответа
    outputs = model.generate(input_ids, eos_token_id=tokenizer.eos_token_id, early_stopping=True)

    # Декодируем ответ и выводим
    decoded_output = tokenizer.decode(outputs[0][1:])  # Отбрасываем начальный токен
    return decoded_output

# Пример
query_text = "Кто устанавливает порядок расчета размера участия иностранного капитала?"
response = generate_response(query_text, top_chunks)
print("Текст ответа:[ " + response + " ]")

Текст запроса:[ Вопрос: Кто устанавливает порядок расчета размера участия иностранного капитала?. Вспомогательный текст: 015 года № 37658 («Вестник Банка России» от 30 апреля 2010 года № 23, от 3 О декабря 201 О года № 73, от 22 июня 2011 года № 3 3, от 28 сентября 2011 года № 54, от 21 декабря 2011 года № 72, от 13 ноября 2013 года № 61, от 20 января 2014 года № 5 - 6, от 22 декабря 2014 года № 112, от 22 июня 2015 года № 54). В целях настоящего Указания под оплаченными акциями (долями) кредитных организаций понимаются: акции кредитной организации, если на дату расчета размера участия иностранного капитала в сово Вспомогательный текст: ытии соответствующего структурного подразделения). Уведомление  о включении требований Банка России в реестр требований кредиторов (направляется конкурсным управляющим в течение трех рабочих дней со дня внесения указанных сведений в реестр требований кредиторов). Тексты публикаций о ходе мероприятий конкурсного производства в отношении фондов, для их ра